# Demo 08 - Probability and Missing Data

In this notebook we'll first take a look at what the effect of missing data can be on a classification mode, specifically using the Titanic dataset. Then we'll look at some ways of "fixing" that missing data, and the effect it has on the model.

In [ ]:
# Only for COLAB!
# clone the course repository, change to right directory, and import libraries.
%cd /content
!git clone https://github.com/nmattei/cmps6790.git
%cd /content/cmps6790/_demos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
# SKLEARN stuff
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
# Load SQLITE
import sqlite3
plt.style.use('fivethirtyeight')
# Make the fonts a little bigger in our graphs.
font = {'size'   : 20}
plt.rc('font', **font)
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
# First let's load up the Titanic dataset and make a decision tree!
# We have a full datasheet here: http://campus.lakeforest.edu/frank/FILES/MLFfiles/Bio150/Titanic/TitanicMETA.pdf
df_titanic = pd.read_csv("./data/titanic.csv")
df_titanic.head(5)

In [ ]:
# We don't need a few of these -- Name, Cabin (why?) and home.dest so let's drop those
# body is the body id number, so that is only there for those that died, so let's drop it too, boat is life boat number
df_titanic.drop(columns=["name","cabin", "ticket","body","boat","home.dest"], inplace=True)
df_titanic.head(5)

In [ ]:
df_titanic.describe()

In [ ]:
# Let's do the easy part first: dummy out sex and embarked, dropna, and go to town.
df_ml = df_titanic.dropna()
df_ml = pd.get_dummies(df_ml, columns=["sex", "embarked"])
df_ml.head(5)

In [ ]:
# Let's MACHINE LEARN!
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
def fit_and_report(df, features, target):
    train, test = train_test_split(df,
                               test_size=0.4,
                               stratify=df[target])
    X_train = train[features]
    y_train = train[target]
    X_test = test[features]
    y_test = test[target]
    mod_dt = DecisionTreeClassifier(max_depth = 3, random_state = 1)
    mod_dt.fit(X_train,y_train)
    prediction=mod_dt.predict(X_test)
    ConfusionMatrixDisplay.from_estimator(mod_dt, X_test, y_test,
                                          display_labels=mod_dt.classes_,
                                          cmap=plt.cm.Blues, normalize='all')
    plt.figure(figsize = (15,8))
    plot_tree(mod_dt, feature_names = features, class_names={1:"survived", 0:"died"}, filled = True);

    print(f"The accuracy of the Decision Tree is {metrics.accuracy_score(prediction,y_test):.3f}")
    print(f"The Precision of the Decision Tree is {metrics.precision_score(prediction,y_test,average='weighted'):.3f}")
    print(f"The Recall of the Decision Tree is {metrics.recall_score(prediction,y_test,average='weighted'):.3f}")

In [ ]:
fit_and_report(df_ml,
                ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_female', 'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S'],
                ["survived"])


In [ ]:
# So we're doing pretty well... but let's see exactly what data is missing..

print(len(df_titanic))

print(len(df_titanic.dropna()))

print(len(df_titanic) - len(df_titanic.dropna()))


In [ ]:
# But what are these features? What's missing? Is it important?
df_titanic[df_titanic.isnull().any(axis=1)]

In [ ]:
# So what kinds of folks are we dropping here?
df_titanic[df_titanic.isnull().any(axis=1)].describe()

In [ ]:
# Where are they from?

print(df_titanic[df_titanic.isnull().any(axis=1)].embarked.value_counts())
print(df_titanic[df_titanic.isnull().any(axis=1)].pclass.value_counts())

In [ ]:
# So it looks like we're missing a ton of south hampton folks, that are in third class...
# Let's impute with the mean for the moment and see how our results change

df_ml_impute = df_titanic.fillna(df_titanic.mean())
df_ml_impute.head(5)

In [ ]:
df_ml_impute.dtypes

In [ ]:
df_ml_impute.describe()

In [ ]:
# Okay, so now how many are we missing?
print(len(df_ml_impute) - len(df_ml_impute.dropna()))

# So we still have 2 missing
display(df_ml_impute[df_ml_impute.isnull().any(axis=1)])

In [ ]:
# Let Machine Learn again!
df_ml_impute = pd.get_dummies(df_ml_impute, columns=["sex", "embarked"])
fit_and_report(df_ml_impute,
                ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_female', 'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S'],
                ["survived"])

In [ ]:
df_titanic["age"].mean()

In [ ]:
df_titanic["age"].median()

In [ ]:
sns.pairplot(df_titanic)

In [ ]:
df_titanic.groupby("sibsp").mean()

## Part 2: The old Notebook


**Note to Nick/Aron**: This notebook is still very TBD -- not sure if we want to focus more on missing data (i.e., not interpreting results correctly) or fuzzywuzzy? Or maybe something else...


This first part is a copy of the missing data notebooks first...


In this notebook we will look at the effects missing data can have on conclusions you can draw from data.  We will also go over some practical implementations for linear regressions in Python

For this work we will be using data from: Generalized body composition prediction equation for men using simple measurement techniques", K.W. Penrose, A.G. Nelson, A.G. Fisher, FACSM, Human Performance research Center, Brigham Young University, Provo, Utah 84602 as listed in Medicine and Science in Sports and Exercise, vol. 17, no. 2, April 1985, p. 189.

[Data availabe here.](http://staff.pubhealth.ku.dk/~tag/Teaching/share/data/Bodyfat.html)

In [ ]:
# Load the Penrose Data
df_penrose = pd.read_csv("./data/bodyfat.csv")

display(df_penrose.head())
# observations = ['Neck', 'Chest', 'Abdomen', 'Hip', 'Thigh', 'Knee', 'Ankle', 'Biceps', 'Forearm', 'Wrist']
observations = ['Age', 'Neck', 'Forearm', 'Wrist']

len(df_penrose)

In [ ]:
# Let's do some really basic scatter plotting...

fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

Let's say we want to look at some linear regressions of single variables to see what is going on!  So let's plot some regression lines.  Note that there are at least a few different ways -- [linregress](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html), [polyfit](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html), and [statsmodels](https://www.statsmodels.org/stable/index.html).

Here's a good article about it [Data science with Python: 8 ways to do linear regression and measure their speed](https://www.freecodecamp.org/news/data-science-with-python-8-ways-to-do-linear-regression-and-measure-their-speed-b5577d75f8b/).

In [ ]:
# Let's do a basic Linear Regression on a Single Variable.
# Note that linregress p-value is whether or not the slope is 0, not if the correlation is significant.
fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):
    slope, intercept, r_value, p_value, std_err = stats.linregress(df_penrose[o],
                                                                   df_penrose['bodyfat'])

    # Pack these into a nice title
    diag_str = "p-value =" + str(round(p_value, 3)) + "\n" + "r-value =" + str(round(r_value, 3)) + "\nstd err. =" + str(round(std_err, 3))
    df_penrose.plot.scatter(x=o, y='bodyfat', title=diag_str, ax=ax[i])

    # Make points and line
    pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
    line = slope * pts + intercept
    ax[i].plot(pts, line, lw=1, color='red')


In [ ]:
# We could also use the polyfit function

# Let's try to fit a linear model with PolyFit.

fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):
    # Fit our curve
    x1, intercept = np.polyfit(df_penrose[o],df_penrose['bodyfat'], 1)

    # Plot regular points
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

    # Plot curve
    pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
    line = x1 * pts + intercept
    ax[i].plot(pts, line, lw=1, ls='-', color='red')

In [ ]:
# Let's try fitting a degree 2 polynomial with polyfit.

fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):

    # Fit the polynomial.
    x2, x1, intercept = np.polyfit(df_penrose[o],df_penrose['bodyfat'], 2)

    # Plot our points.
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

    # Plot the Regression Line..
    pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
    line = x2 * pts**2 + x1 * pts + intercept
    ax[i].plot(pts, line, lw=1, ls='-', color='red')

In [ ]:
# Let's try fitting a degree 5 polynomial with polyfit.

fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):

    # Fit the polynomial.
    x5, x4, x3, x2, x1, intercept = np.polyfit(df_penrose[o],df_penrose['bodyfat'], 5)

    # Plot our points.
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

    # Plot the Regression Line..
    pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
    line = x5 * pts**5 + x4 * pts**4 + x3 * pts**3 + x2 * pts**2 + x1 * pts + intercept
    ax[i].plot(pts, line, lw=1, ls='-', color='red')

### A More Complicated example with Statsmodels.

Statsmodels (you'll likely need to install it) gives a much more R-like interface to linear modeling.  You can read [more about it here](https://www.statsmodels.org/stable/index.html).

In [ ]:
import statsmodels.api as sm
df_ind = df_penrose[['Neck', 'Wrist']]
df_target = df_penrose['bodyfat']

In [ ]:
X = df_ind
y = df_target

# Note the difference in argument order
# Call: endog, then exog (dependent, indepenednt)
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model
# Print out the statistics
model.summary()
#fig, ax = plt.subplots(figsize=(12,8))
#fig = sm.graphics.plot_partregress(endog="bodyfat", exog_i=['Abdomen', 'Neck'], exog_others='', data=df_penrose)

We can also use the [single regressor plot](https://tedboy.github.io/statsmodels_doc/generated/statsmodels.graphics.api.plot_partregress.html#statsmodels.graphics.api.plot_partregress).

In [ ]:
from statsmodels.graphics.regressionplots import plot_partregress
fig, ax = plt.subplots(figsize=(12,8))
plot_partregress(endog='bodyfat', exog_i='Neck', exog_others='', data=df_penrose, ax=ax)
plt.show()

If we have multiple elements in our regression then we need to use a different plot.

In [ ]:
# Multiple regression plot
from statsmodels.graphics.regressionplots import plot_partregress_grid
fig = plt.figure(figsize=(8, 6))
plot_partregress_grid(model, fig=fig)
plt.show()

Another way to work with regressions and their plots is using the [Seaborn Regression Package](https://seaborn.pydata.org/tutorial/regression.html)

In [ ]:
# Another way to do simple exploratory plots
import seaborn as sns
df_test = df_penrose.sample(frac=0.10, replace=False)
fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):
    sns.regplot(x=o, y='bodyfat', data=df_test, ax=ax[i])
    #g.axes.set_xlim(df_test[o].min()*.95,df_test[o].max()*1.05)

Another nice simulator to play with is [this one](https://ndirienzo.shinyapps.io/linear_regression_sim/) which is from [Prof. Nicholas DiRienzo](https://ischool.arizona.edu/people/nicholas-dirienzo) from ASU's School of Information

## Logistic Regression

We can use sklearn to do a quick logistic regression.  Remember that for logistic regression we are testing whether or not something is true, so we need to add a variable to our data.

Someone is obese if their body fat is >32% so we'll add a dummy for that!

In [ ]:
df_penrose['obese'] = df_penrose.apply(lambda x: 1 if x['bodyfat'] > 32 else 0, axis=1)

In [ ]:
df_penrose.head(5)

In [ ]:
# We're going to use sklearn to build us a classifier.

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# setup our data for testing and training.

X_train, X_test, y_train, y_test = train_test_split(df_penrose[observations],
                                                    df_penrose['obese'],
                                                    test_size=0.2)


In [ ]:
X_train[:10]

In [ ]:
# Fit that model!
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced')
model = logisticRegr.fit(X_train, y_train)

In [ ]:
# Fit and plot!
y_pred = model.predict(X_test)
print(f"Accuracy Score is: {accuracy_score(y_test, y_pred)}")
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test,
                                          display_labels=model.classes_,
                                          cmap=plt.cm.Blues, normalize='all')


# Now back to Missing Data!!

What happens if we start to remove parts of the data -- is the relationship still as strong?

We can use the [pandas sample command](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) to remove some of the dataframe.

Note that here we are just asking the question, if we took some of the data out randomly, do we still get the same result?

In [ ]:
# Let's do a basic Linear Regression on a Single Variable.
# Note that linregress p-value for the null-hyp that slope = 0.
df_test = df_penrose.sample(frac=0.2, replace=False)

fig, ax = plt.subplots(1, 4, figsize=(15,5))
for i,o in enumerate(observations):
    slope, intercept, r_value, p_value, std_err = stats.linregress(df_test[o],
                                                                   df_test['bodyfat'])

    # Pack these into a nice title
    diag_str = "p-value =" + str(round(p_value, 7)) + "\n" + "r-value =" + str(round(r_value, 7)) + "\nstd err. =" + str(round(std_err, 7))
    df_test.plot.scatter(x=o, y='bodyfat', title=diag_str, ax=ax[i])

    # Make points and line
    pts = np.linspace(df_test[o].min(), df_test[o].max(), 500)
    line = slope * pts + intercept
    ax[i].plot(pts, line, lw=1, color='red')

If we want to determine if these correlations are significant under the missing data then we need to run bootstrap samples and see what happens.

Nick -- modify this to drop part of the data then resample from the dropped part!

In [ ]:
results = {o:[] for o in observations}
bootstrap_samples = 1000
fraction = 0.20

for i,o in enumerate(observations):
    for t in range(bootstrap_samples):
        df_test = df_penrose.sample(frac=fraction, replace=False)
        slope, intercept, r_value, p_value, std_err = stats.linregress(df_test[o],df_test['bodyfat'])
        #r,p = stats.pearsonr(df_test[o], df_test['bodyfat'])
        results[o].append(p_value)

rs = pd.DataFrame(results)
ax = rs.boxplot()
ax.set_ylim([-0.01,0.30])
ax.set_title(f"p-value of 1 variable regression over {bootstrap_samples} iterations")
ax.set_ylabel("p-value")
ax.set_xlabel("Body Part")
ax.axhline(y=0.05, lw=2, color='red')
plt.show()

As we can see above as we run more and more samples and plot the p-values

# 11 - Probability and Simulation

In this notebook we look at some fun ways to do sampling and test some of the basics of probability just for giggles.

## Probability and Code!

Note we're using [Numpy's probability functions](https://numpy.org/doc/stable/reference/random/index.html), you could also use [Python's](https://docs.python.org/3/library/random.html)

In [ ]:
# Let's make a probability distribution:
outcomes = list(range(1,7))
outcomes

In [ ]:
#Simulate an outcome..
np.random.choice(outcomes)

In [ ]:
# Do it a lot...
np.random.choice(outcomes, 20)

In [ ]:
# Graph it!
results = pd.DataFrame(np.random.choice(outcomes, 1000))
results.plot.hist(bins=np.arange(0.5,7.5, 1))

In [ ]:
# Do it with a biased coin..
b = 1.0 / 7.0
b1 = 2.0 / 7.0
results = pd.DataFrame(np.random.choice(outcomes, 1000, p=[b, b, b1, b, b, b]))
results.plot.hist(bins=np.arange(0.5,7.5, 1))

In [ ]:
# Do it for multiple events!
die1 = np.random.choice(outcomes, 10000)
die2 = np.random.choice(outcomes, 10000)
results = pd.DataFrame({'Die1': die1, 'Die2':die2})
results.head()

In [ ]:
# Need to add them up...
plt.figure(figsize = (12,5))
results['sum'] = results["Die1"] + results["Die2"]
results['sum'].plot.hist(bins=np.arange(1.5, 13.5, 1), density=True)

In [ ]:
# Default is with replacement but we can do without replacement..
people = ['Winona', 'Xanthippe', 'Yvonne', 'Zelda']
np.random.choice(people, 3, replace=False)

### One thing we may want to do in a stats model is ....

See if a particular distribution is the same as some known distribution. To do this we typically use the [Chi Squared test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html) if we known the underlying distribution.

Here we know that rolling two dice and summing them **should** give us a normal distribution so we can use a more complex [normal test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html) from Pearson (of correlation coefficient fame) to check...

**Returns**

statistic : float or array
s^2 + k^2, where s is the z-score returned by skewtest and k is the z-score returned by kurtosistest.

pvalue: float or array
A 2-sided chi squared probability for the hypothesis test.

In [ ]:
# See if one die is really a uniform distribution.
#
# Note that we need: f_obsarray_like
#                    Observed frequencies in each category.
#                    f_exparray_like, optional
#Expected frequencies in each category. By default the categories are assumed to be equally likely.
#

from scipy import stats
chisq, p = stats.chisquare(results["Die1"].value_counts(), [1./6, 1./6, 1./6, 1./6, 1./6, 1./6])
print("ChiSq = {} and p = {}".format(chisq,p))

In [ ]:
# See if the sum is normal..

from scipy import stats
k2, p = stats.normaltest(results["sum"])
print("K2 = {} and p = {}".format(k2,p))

What could things like this be used for when building statistical models? [Hint!](http://data8.org/materials-sp18/lec/lec16PDF.pdf)

## Looking at Two Variables.

Let's roll two dice a bunch of times and see the resutls.


In [ ]:
die1 = np.random.choice(outcomes, 100)
die2 = np.random.choice(outcomes, 100)
results = pd.DataFrame({'Die1': die1, 'Die2':die2})

In [ ]:
counts = pd.crosstab(results['Die1'], results['Die2'])
counts

In [ ]:
joint = pd.crosstab(results['Die1'], results['Die2'], normalize=True)
joint

In [ ]:
# Now we can roll this up for either die to see it's distribution
joint.sum(axis=0)

In [ ]:
# Can also get marginals directly.
marginals = pd.crosstab(results['Die1'], results['Die2'], normalize=True, margins=True)
marginals

In [ ]:
# Finally, if we want conditional distributions we have to do a bit of work. Let's try to work out
# P(Die 1 is a 6 | Die 2 is a 5)

counts = pd.crosstab(results['Die1'], results['Die2'])
counts

In [ ]:
# We need to get the (Die 2 is a 5 row) and then look at the distribution there..

counts[5] / counts[5].sum()

## Using Simulation to Answer Probability Questions.

In CMPS 2170 we figured out closed form formulas for a set of mutually independent Bernoilli Trials.

* Bernoulli Trial: an experiment with two possible outcomes
* E.g., flip a coin results in two possible outcomes: head (𝐻) and tail (𝑇)
* Independent Bernoulli Trials: a sequence of Bernoulli trails that are mutually independent

* Example: What is the probability of the sequence HHHTT for a coin flip sequence with $p$ for H and $1-p$ for T?
  * $p^3(1-p)^2$.

Recall: The probability of exactly $k$ successes in $n$ independent Bernoulli trials, with probability of success $p$ and probability of failure $q = 1 − p$, is $C(n,k)p^kq^{n-k}$ where $C(n,k)$ is $n$ choose $k$.

In [ ]:
# Setup a biased coin and flip it a bunch..
coin_results = np.random.choice(["Heads", "Tails"], 100, p=[0.75, 0.25])
coin_results

## A More complex Question..

* What is the probability of getting 60 or more heads if I flip 100 coins?
* Approximation through simulation:
  1. Figure out how to do one experiment (i.e., flip 100 coins).
  2. Run the experiment a bunch of times.
  3. Find the fraction of times where number of heads >= 60.

In [ ]:
# Flip 100 coins and count heads...
coin_results = np.random.choice(["Heads", "Tails"], 100, p=[0.75, 0.25])
print(coin_results == 'Heads')
print(np.count_nonzero(coin_results == 'Heads'))


In [ ]:
# Wrap it up and do it a bunch...
# Note we're using Numpy here for broadcasting -- numpy arrays are imuteable so
# it's a tad more akaward in places..
n_reps = 10000

def exp():
    coin_results = np.random.choice(["Heads", "Tails"], 100, p=[0.50, 0.50])
    return np.count_nonzero(coin_results == 'Heads')

head_counts = np.array([])
for i in range(n_reps):
    head_counts = np.append(head_counts, exp())

In [ ]:
# Figure it out...
print(np.count_nonzero(head_counts >= 60))
print(np.count_nonzero(head_counts >= 60) / n_reps)

If we work out the math we need at least 60 H so we have to add up quite a few things...
$\sum^{100}_{k=60} C(100, k)p^kq^n-k$

In [ ]:
# Using simiulation we can also look at the trials
n_reps = 1000

head_counts = np.array([])
for i in range(n_reps):
    head_counts = np.append(head_counts, exp())

results = pd.DataFrame(head_counts)
ax = results.plot.hist()
plt.xlim(20,70)
plt.axvline(np.mean(head_counts), color='red')
plt.title(f"Mean: {np.mean(head_counts)}")
plt.show()

## Settle the Monty Hall Thing...

In [ ]:
def simulate_monty_hall():
    behind_picked_door = np.random.choice(['Car', 'Goat 1', 'Goat 2'])

    if behind_picked_door == 'Car':
        winning_strategy = 'Stay'
    else:
        winning_strategy = 'Switch'

    print(behind_picked_door, 'was behind the door. Winning strategy:', winning_strategy)
    return winning_strategy
simulate_monty_hall()

In [ ]:
# Run it a bunch...
n_repetitions = 10000

winning_strategies = np.array([])
for i in np.arange(n_repetitions):
    winning_strategy = simulate_monty_hall()
    winning_strategies = np.append(winning_strategies, winning_strategy)


In [ ]:
np.count_nonzero(winning_strategies == 'Switch') / n_repetitions

In [ ]:
np.count_nonzero(winning_strategies == 'Stay') / n_repetitions